In [1]:
import pandas as pd
import numpy as np

In [2]:
df_net= pd.read_csv('transformacion_netflix.csv', delimiter=',', encoding='utf-8')
df_dis= pd.read_csv('transformacion_disney.csv', delimiter=',', encoding='utf-8')
df_hulu= pd.read_csv('transformacion_hulu.csv', delimiter=',', encoding='utf-8')
df_amazon= pd.read_csv('transformacion_amazon.csv', delimiter=',', encoding='utf-8')



In [3]:
#creo un archivo gral
df_plataformagral = pd.concat([df_net,df_hulu,df_amazon,df_dis])

In [ ]:
#lo guardo
df_plataformagral.to_csv('totalplataformas.csv', index=False)

In [3]:
df_plataformagral = pd.read_csv('totalplataformas.csv', delimiter=',', encoding='utf-8')

Película (sólo película, no serie, etc) con mayor duración según año, plataforma y tipo de duración. La función debe llamarse get_max_duration(year, platform, duration_type) y debe devolver sólo la cadena del nombre de la película.

In [5]:
#pruevo filtros
clasificacion = df_plataformagral[ (df_plataformagral.type=='movie')
& (df_plataformagral.listed_in!='documentaries')]
clasificacion.head(2)

,id,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,duration_int,duration_type
6,ns7,s7,movie,my little pony: a new generation,"robert cullen, josé luis ucha","vanessa hudgens, kimiko glenn, james marsden, ...",NaN,2021-09-24,2021,pg,91 min,children & family movies,equestria's divided. but a bright-eyed hero be...,91.0,min
7,ns8,s8,movie,sankofa,haile gerima,"kofi ghanaba, oyafunmike ogunlano, alexandra d...","united states, ghana, burkina faso, united kin...",2021-09-24,1993,tv-ma,125 min,"dramas, independent movies, international movies","on a photo shoot in ghana, an american model s...",125.0,min


In [4]:

def get_max_duration(year:int,platform:str,duration_type:str):
    
    #declaro parametros
    if platform == 'netflix':
        inicioId = 'n'
    elif platform == 'hulu':
        inicioId = 'h'
    elif platform == 'disney':
        inicioId = 'd'
    elif platform == 'amazon':
        inicioId = 'a'
    else:
        return 'no se encuentra plataforma, ingrear una de las opciones: netflix, hulu, disney o amazon'
    

    if duration_type == 'pelicula':
        durationtype= 'movie' 
    else:
        return 'ingresar parametro : pelicula' 



    filtro = df_plataformagral[ (df_plataformagral.type==durationtype) #filtro por peliculas
                            & (df_plataformagral.id.str.contains(inicioId)) #filtro por primer letra de plataforma
                            & (df_plataformagral.release_year==year)]  #filtro por el año

    max = filtro['duration_int'].max() #guardo el valor maximo
    filtro1 = filtro.query("duration_int==@max") #consulto la duracion de ese valor
    pelicula = filtro1['title'] #guardo lo que quiero que retorne
    if  not pelicula.empty: #si segun los datos ingresados el dataframe no esta vacio
        return {'pelicula': pelicula.iloc[0]}
    else:
        return 'Los datos ingresados no se encuentran en la base de datos'
    

In [5]:
get_max_duration(2021,'disney', 'pelicula') #verifico funcion

{'pelicula': 'cruella'}

Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año 

Cantidad de películas (sólo películas, no series, etc.) según plataforma, con un puntaje mayor a XX en determinado año. La función debe llamarse get_score_count(plataforma, anotado, año) y debe devolver un int, con el total de películas que cumplen con lo solicitado.

In [6]:
df_score= pd.read_csv('score_movies.csv', delimiter=',', encoding='utf-8')
#abro archivo solo con el score promedio por peliculas

In [8]:
#voy filtrando y verificando los algoritmos
clasificacion = df_plataformagral[ (df_plataformagral.type=='movie')#busco  los id solo de las peliculas
                                    & (df_plataformagral.id.str.contains('n')) #filtro por plataforma
                                     & (df_plataformagral.release_year==2015)] #filtro por año
consulta_score= df_score[(df_score.movieId.str.contains('n'))]#filtro por plataforma
df_peliculas=consulta_score[consulta_score.movieId.isin(clasificacion.id)]#me quedo solo con los id que estan en ambos df
total_peliculas= df_peliculas.score[df_peliculas.score > 1].count()#filtro por el puntaje mayor a xx del campo score y cuento los valores totales
total_peliculas

398

In [8]:
def get_score_count(platform, scored, year):  

    import datetime

    #declaro parametros
    if platform == 'netflix':
        plataforma = 'n'
    elif platform == 'hulu':
        plataforma = 'h'
    elif platform == 'disney':
        plataforma = 'd'
    elif platform == 'amazon':
        plataforma = 'a'
    else:
        return 'ingresar una de las opciones: netflix, hulu, amazon o disney'
    
    
    clasificacion = df_plataformagral[ (df_plataformagral.type=='movie')#busco  solo las peliculas
                                        & (df_plataformagral.id.str.contains(plataforma)) #filtro por plataforma
                                        & (df_plataformagral.release_year==year)]  #filtro por el año
                                        

    consulta_score= df_score[(df_score.movieId.str.contains(plataforma))] #filtro por plataforma 
    df_peliculas=consulta_score[consulta_score.movieId.isin(clasificacion.id)]#me quedo solo con los id que estan en ambos df
    total_peliculas = df_peliculas.score[df_peliculas.score >scored].count()#filtro por el puntaje mayor a xx del campo score y cuento los valores totales


    return {'plataforma' : platform , 'cantidad': total_peliculas, 'anio' : year, 'score' : scored}

In [9]:
get_score_count('netflix', 1, 2015)

{'plataforma': 'netflix', 'cantidad': 398, 'anio': 2015, 'score': 1}

Cantidad de películas (sólo películas, no series, etc) según plataforma. La función debe llamarse get_count_platform(platform) y debe devolver un int, con el número total de películas de esa plataforma. Las plataformas deben llamarse amazon, netflix, hulu, disney.

In [10]:
def get_count_platform(platform): 

    #Declaro parametros
    if platform == 'netflix':
        plataforma = 'n'
    elif platform == 'hulu':
        plataforma= 'h'
    elif platform == 'disney':
        plataforma= 'd'
    elif platform == 'amazon':
        plataforma= 'a'
    else:
        return print('ingrese plataforma: amazon, disney, hulu o netflix')

   

    consulta_plataforma= df_plataformagral[(df_plataformagral.id.str.contains(plataforma))#filtro por plataforma
                                           & (df_plataformagral.type=='movie')]#filtro por peliculas

                                           
    conteo_movies= consulta_plataforma.title.count()#cuento
    return { 'plataforma': platform, 'peliculas': conteo_movies}

In [11]:
get_count_platform('netflix')

{'plataforma': 'netflix', 'peliculas': 6131}

Actor que más se repite según plataforma y año. La función debe llamarse get_actor(platform, year) y debe devolver sólo la cadena con el nombre del actor que más se repite según la plataforma y el año dado.

In [231]:
#verifico algoritmos
filtroplat = df_plataformagral[(df_plataformagral.id.str.contains('n'))
                                & (df_plataformagral.release_year==2015)] #filtro por año y plataforma
lista= filtroplat['cast']
newlista = [x for x in lista if pd.isnull(x) == False]
lista_cast=[]
for i in newlista: #itero para separar actores en diferentes str
    separador= i.split(sep=',')
    for j in separador: #itero para eliminar espacios en cada nuevo str y agregar a lista
        valores=j.strip()
        lista_cast.append(valores) #agrego valores a lista_cast
    df_lista_cast= pd.Series(lista_cast) #transformo a dataframe
df_lista_cast.value_counts().max()

5

In [12]:
def get_actor(platform, year): 

    #declaro parametros
    if platform == 'netflix':
        plataforma= 'n'
    elif platform == 'disney':
        plataforma= 'd'
    elif platform == 'hulu':
        plataforma= 'h'
    elif platform == 'amazon':
        plataforma= 'a'
    else:
        return 'ingresar plataforma valida: amazon, disney, hulu o netflix'


    filtroplat = df_plataformagral[(df_plataformagral.id.str.contains(plataforma))
                                   & (df_plataformagral.release_year==year)] #filtro por año y plataforma

    if filtroplat.empty: #si devuelve df vacio
        
        return 'el año no se encuentra en el dataset'
    else:
        lista= filtroplat['cast'] #guardo en nueva variable los actores por peliculas
        newlista = [x for x in lista if pd.isnull(x) == False]#elimino flotantes

        lista_cast=[]
        for i in newlista: #itero para separar actores en diferentes str
            separador= i.split(sep=',')
            for j in separador: #itero para eliminar espacios en cada nuevo str y agregar a lista
                    valores=j.strip()
                    lista_cast.append(valores) #agrego valores a lista_cast
        df_lista_cast= pd.Series(lista_cast) #transformo a dataframe
        return {'plataforma': platform, 'anio': year, 'actor' : df_lista_cast.value_counts().idxmax(), 'apariciones' : df_lista_cast.value_counts().max()} #imprimo copnsulta del str mas repetido

In [13]:
get_actor('netflix', 2015)

{'plataforma': 'netflix',
 'anio': 2015,
 'actor': 'james franco',
 'apariciones': 5}

La cantidad de contenidos/productos (todo lo disponible en streaming) que se publicó por país y año. La función debe llamarse prod_per_county(tipo,pais,anio) deberia devolver la cantidada de contenidos/productos segun el tipo de contenido (pelicula,serie) por pais y año en un diccionario con las variables llamadas 'pais' (nombre del pais), 'anio' (año), 'pelicula' (cantidad de contenidos/productos).

In [33]:
pais= 'canada'

In [35]:
prueba= filtro_anioytipo[filtro_anioytipo.country.str.contains(pais)]
prueba.type.count()

38

In [37]:
prueba.shape

(38, 15)

In [30]:
filtro_anioytipo=df_plataformagral[(df_plataformagral.type=='movie') 
                                    & (df_plataformagral.release_year==2015)
                                    & (df_plataformagral.country.notnull())]


In [38]:
df_plataformagral.isna().sum()

id                   0
show_id              0
type                 0
title                0
director          8259
cast              5321
country          11499
date_added       10982
release_year         0
rating               0
duration           482
listed_in            0
description          4
duration_int       482
duration_type      482
dtype: int64

In [54]:
def prod_per_county(tipo,pais,anio):

    #declaramos parametros
    
    if tipo == 'peliculas' :
        variable_tipo= 'movie'
    elif tipo == 'series':
        variable_tipo= 'tv show'
    else:
        return 'ingresar parametro : peliculas o series'

    filtro=df_plataformagral[(df_plataformagral.type=='movie') #filtro solo peliculas
                                    & (df_plataformagral.release_year==anio) #filtro por año
                                    & (df_plataformagral.country.notnull())#quitamos los nulos
                                    & (df_plataformagral.country.str.contains(pais))]#filtramos por el pais ingresado

    respuesta= filtro.type.count()#contamos el total del contenido

    return {'pais': pais, 'anio': anio, variable_tipo: respuesta}

In [68]:
prod_per_county('peliculas', 'canada', 2020)

{'pais': 'canada', 'anio': 2020, 'movie': 30}

La cantidad total de contenidos/productos (todo lo disponible en streaming, series, peliculas, etc) según el rating de audiencia dado (para que publico fue clasificada la pelicula). La función debe llamarse get_contents(rating) y debe devolver el numero total de contenido con ese rating de audiencias.

In [81]:
#realizo pruevas
rating= 'g'
df_plataformagral.type[df_plataformagral.rating==rating].count()

1269

In [10]:
#verificamos los valores unicos del campor rating
df_plataformagral.rating.unique()

array(['pg-13', 'tv-ma', 'pg', 'tv-14', 'tv-pg', 'tv-y', 'tv-y7', 'r',
       'tv-g', 'g', 'nc-17', '74 min', '84 min', '66 min', 'nr',
       'tv-y7-fv', 'ur', 'not rated', '2 seasons', '93 min', '4 seasons',
       '136 min', '91 min', '85 min', '98 min', '89 min', '94 min',
       '86 min', '3 seasons', '121 min', '88 min', '101 min', '1 season',
       '83 min', '100 min', '95 min', '92 min', '96 min', '109 min',
       '99 min', '75 min', '87 min', '67 min', '104 min', '107 min',
       '103 min', '105 min', '119 min', '114 min', '82 min', '90 min',
       '130 min', '110 min', '80 min', '6 seasons', '97 min', '111 min',
       '81 min', '49 min', '45 min', '41 min', '73 min', '40 min',
       '36 min', '39 min', '34 min', '47 min', '65 min', '37 min',
       '78 min', '102 min', '129 min', '115 min', '112 min', '61 min',
       '106 min', '76 min', '77 min', '79 min', '157 min', '28 min',
       '64 min', '7 min', '5 min', '6 min', '127 min', '142 min',
       '108 min', '57 min'

In [95]:
def get_contents(rating):
    
    total_contenido= df_plataformagral.type[df_plataformagral.rating==rating].count()


    if total_contenido == 0 :
        return {'ingresar una de las sigientes categorias ' : '''pg-13', 'tv-ma', 'pg', 'tv-14', 'tv-pg', 'tv-y', 'tv-y7', 'r','tv-g', 'g', 'nc-17', 'nr','tv-y7-fv', 'ur', 'not rated','13+', 'all', '18+','16+', '7+', 'tv-nr', 'unrated', '16','ages_16_', 'ages_18_', 'all_ages', 'not_rate'''}
    else:
        return {'rating': rating, 'contenido' : total_contenido}


In [97]:
get_contents('g')

{'rating': 'g', 'contenido': 1269}